In [1]:
import duckdb
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

#### Review a snapshot data file using `duckdb`

In [14]:
duckdb.sql("install 'httpfs'; load 'httpfs'; set s3_region='us-east-1'; install json; load json;")

In [3]:
duckdb.sql(
"""
    select
        count(*)
    from read_json_auto(
        'https://openalex.s3.amazonaws.com/data/authors/updated_date%3D2023-07-21/part_000.gz',
        format='newline_delimited',
        compression='gzip'
    )
"""
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        40825 │
└──────────────┘

In [34]:
duckdb.sql(
    """
    with unnest_counts_by_year as (    
        select
           id,
           unnest(counts_by_year, recursive := true)
        from read_json(
           'https://openalex.s3.amazonaws.com/data/authors/updated_date%3D2023-07-21/part_000.gz',
           format='newline_delimited',
           auto_detect = true,
           compression='gzip')
    )
    
    select count(*), count(distinct id) from unnest_counts_by_year;
    """
)

┌──────────────┬────────────────────┐
│ count_star() │ count(DISTINCT id) │
│    int64     │       int64        │
├──────────────┼────────────────────┤
│        28837 │              20865 │
└──────────────┴────────────────────┘

#### Explore a small sample of the records to explore the data model

In [35]:
df = duckdb.sql(
    """
        select
            *
        from read_json_auto(
            'https://openalex.s3.amazonaws.com/data/authors/updated_date%3D2023-07-21/part_000.gz',
            format='newline_delimited',
            compression='gzip'
        )
        limit 100
    """).df()

In [36]:
df.columns

Index(['id', 'orcid', 'display_name', 'display_name_alternatives',
       'works_count', 'cited_by_count', 'most_cited_work', 'summary_stats',
       'ids', 'last_known_institution', 'counts_by_year', 'x_concepts',
       'works_api_url', 'updated_date', 'created_date', 'updated'],
      dtype='object')

In [ ]:
df

In [19]:
duckdb.sql("""
    select
        count(*)
    from read_json_auto(
        '/Users/m20/projects/open_alex_authors/data/authors/*/*.gz',
        format='newline_delimited',
        compression='gzip'
    )
"""
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     92840725 │
└──────────────┘

In [ ]:
!aws s3 ls --summarize --human-readable --no-sign-request --recursive "s3://openalex/data/authors/"

##### Filtering IDs of authors who have been cited (`where cited_by_count != 0`)

In [ ]:
con = duckdb.connect('open_alex_authors.duckdb')
con.sql("select id from open_alex_authors.september_2023_snapshot where cited_by_count != 0")

##### Unnesting author records by year

In [ ]:
con.sql(
    """
        select
            id as author_id,
            unnest(counts_by_year, recursive := true)
        from open_alex_authors.september_2023_snapshot
    """
)

##### Getting the `display_name` from `last_known_institution`

In [ ]:
con.sql("""
    select
        id,
        display_name,
        json_extract(
            last_known_institution, '$.display_name'
        ), 
        works_count, 
        cited_by_count 
    from open_alex_authors.september_2023_snapshot
    where cited_by_count > 10
""")

In [12]:
con.close()

In [41]:
duckdb.sql("""
    with counts_by_year as (
    select
        counts_by_year
    from read_json(
        'https://openalex.s3.amazonaws.com/data/authors/updated_date%3D2023-07-21/part_000.gz',
        columns = {
           id: 'VARCHAR',
           orcid: 'VARCHAR',
           display_name: 'VARCHAR',
           works_count: 'INT64',
           cited_by_count: 'INT64',
           most_cited_work: 'VARCHAR',
           counts_by_year: 'JSON',
           x_concepts: 'JSON',
           works_api_url: 'VARCHAR',
           updated_date: 'DATE',
           created_date: 'DATE',
           updated: 'DATE'
        },
        format='newline_delimited',
        compression='gzip'
        )
    )
           
    select
        counts_by_year -> '$.cited_by_count'
    from counts_by_year   
    """
)

┌────────────────────────────────────────┐
│ (counts_by_year -> '$.cited_by_count') │
│                  json                  │
├────────────────────────────────────────┤
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│  ·                                     │
│  ·                                     │
│  ·                                     │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL                                   │
│ NULL     

In [42]:
duckdb.sql(
    """
    create table concepts as 
    select
        *
    from read_json_auto('s3://openalex/data/concepts/*/*.gz',
    compression = 'gzip',
    format = 'newline_delimited')
    """
)

┌──────────────────────┬──────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│          id          │       wikidata       │ … │ updated_date │ created_date │       updated        │
│       varchar        │       varchar        │   │     date     │     date     │       varchar        │
├──────────────────────┼──────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ https://openalex.o…  │ https://www.wikida…  │ … │ 2023-06-07   │ 2016-06-24   │ 2023-06-07           │
│ https://openalex.o…  │ https://www.wikida…  │ … │ 2023-06-07   │ 2016-06-24   │ 2023-06-07           │
│ https://openalex.o…  │ https://www.wikida…  │ … │ 2023-06-07   │ 2016-06-24   │ 2023-06-07           │
│ https://openalex.o…  │ https://www.wikida…  │ … │ 2023-06-07   │ 2016-06-24   │ 2023-06-07           │
│ https://openalex.o…  │ https://www.wikida…  │ … │ 2023-06-07   │ 2016-06-24   │ 2023-06-07           │
│ https://openalex.o…  │ https://www.wikida…  │ … │ 202

In [48]:
df = duckdb.sql(
    """
    select
        *
    from read_json_auto('./data/authors/updated_date=2023-08-*/*.gz',
    format = 'newline_delimited',
    compression = 'gzip'
    );
    """
).df()

: 

: 